# Closed loop laser scan with NI-DAQmx

## Setup

In [1]:
import time
import numpy as np
import socket

from pylabnet.utils.logging.logger import LogClient
from pylabnet.scripts.lasers import wlm_monitor
from pylabnet.network.client_server import sweeper, si_tt
from pylabnet.scripts.sweeper.sweeper import Sweep1D, MultiChSweep1D
from pylabnet.utils.helper_methods import create_server

Enter host and port parameters here:

In [2]:
logger_host = '140.247.189.94'
logger_port = 6265
wavemeter_host = '140.247.189.94'
wavemeter_port = 8298
ctr_host = '140.247.189.82'
ctr_port = 48067

Instantiate all clients

In [3]:
exp_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop'
)
wm = wlm_monitor.Client(
    host=wavemeter_host,
    port=wavemeter_port,
)
ctr = si_tt.Client(
    host=ctr_host,
    port=ctr_port
)
ctr.start_rate_monitor(name='sweeper', ch_list=[8])
exp = Sweep1D(logger=exp_log)

In [4]:
update_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='1D_update_server'
)
pause_1d_service = sweeper.Service()
pause_1d_service.assign_module(module=exp)
pause_1d_service.assign_logger(logger=update_log)
pause_1d_server, pause_1d_port = create_server(
    service=pause_1d_service,
    logger=update_log,
    host=socket.gethostbyname_ex(socket.gethostname())[2][0]
)
update_log.update_data(data={'port': pause_1d_port})
pause_1d_server.start()

## Experiment

Parameters, setup

In [5]:
f0 = 406.568
f1 = 406.574
pts =401
exp.set_parameters(min=f0, max=f1, pts=pts)

Experiment script

In [22]:
# wm.update_parameters([{
#         'channel': 7, 'min_voltage': 4
#     }])

In [6]:
def experiment(frequency):

    # Set frequency
    wm.update_parameters([{
        'channel': 6, 'setpoint': frequency
    }])

    return ctr.get_count_rate(name='sweeper', integration=0.1)
    

In [7]:
exp.configure_experiment(experiment)

In [10]:
exp.set_reps(reps=0)
exp.run(plot=True, autosave=True, filename='Sasha_on_scan')

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'f31312e5-48d5-4f84-b0ea-76e4b9f4869b',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'e5d806fb-3737-42a3-982d-d1b23d55c9c5',
 …

Finished 1 out of 2 sweeps.
Finished 2 out of 2 sweeps.


In [ ]:
f0 = 406.565
f1 = 406.75
pts = 201
exp.set_parameters(min=f0, max=f1, pts=pts)

## Multi-channel sweep

In [9]:
mc_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop_multichannel'
)

mc_pause_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='MultiCh1D_pause_server'
)

m_exp = MultiChSweep1D(logger=mc_log, channels=['Transmission', 'Reflection'])
pause_m_service = sweeper.Service()
pause_m_service.assign_module(module=exp)
pause_m_service.assign_logger(logger=mc_pause_log)
pause_m_server = create_server(
    service=pause_m_service,
    logger=mc_pause_log,
    host=logger_host
)

In [18]:
m_exp.set_parameters(min=f0, max=f1, pts=pts, reps=4)

In [19]:
def multi_experiment(frequency):
    time.sleep(0.02)
    diffusion = 0.0002*np.sin(2*np.pi*time.time()/50)
    return [
        100 + 0.01/((frequency-406.694+diffusion)**2+(0.0001)**2),
        100 - 0.01/((frequency-406.694+diffusion)**2+(0.0001)**2)
    ]
m_exp.configure_experiment(multi_experiment)

In [20]:
m_exp.run(plot=True)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Transmission Single',
              'type…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'a36df67b-3158-49db-a510-0af4474ca83f',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Transmission Single',
              'type…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '9edc977a-61b7-4b8e-ba4b-46b5c4b7e1f6',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Reflection Single',
              'type':…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '1825d0c6-bb91-4049-97b0-de0e9d2c5364',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Reflection Single',
              'type':…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'b8187589-e1e3-46f5-ba41-4f153018ad26',
 …

## Close counter

In [5]:
#ni.close_timed_counter('counter_0')